In [34]:
import torch
import random
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing

#stock network import
import torchvision.models as models

from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader

In [35]:
#Setting GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#define model
model = models.resnet18(weights = None)
num_ftrs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Dropout(0.5),  # Dropout layer
    nn.Linear(num_ftrs, 10)  # Fully connected layer
)

#load trained model
model.load_state_dict(torch.load('/school/intelligence_coursework/Trained_networks/model_state_dict_2.pth'))

#Using GPU
model = model.to(device)
print("Model device:", next(model.parameters()).device)

# Make sure to set 'spawn' as the start method
multiprocessing.set_start_method('spawn', force=True)

Model device: cuda:0


In [36]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)), # Random crop back to 32x32
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizing the images
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=False, num_workers=4)

classes = trainset.classes
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
checkpoint = torch.load('/school/intelligence_coursework/Trained_networks/model_clpso_checkpoint_20_swarm.pth')
model.load_state_dict(checkpoint['model_state_dict'])
particles = checkpoint['particles']
velocities = checkpoint['velocities']
personal_best_positions = checkpoint['personal_best_positions']
personal_best_scores = checkpoint['personal_best_scores']
global_best_position = checkpoint['global_best_position']
global_best_score = checkpoint['global_best_score']
epoch_losses = checkpoint['epoch_losses']
epoch_precisions = checkpoint['epoch_precisions']
model.train()  # Set the model to training mode


In [37]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [ ]:
#freeze network
for param in model.parameters():
    param.requires_grad = False

    #unfreeze fuly connected layer
for param in model.fc.parameters():
    param.requires_grad = True

#randomize the wights
model.fc[1].reset_parameters()

# Initialize early stopping
early_stopping = EarlyStopping(patience=5)

#validation loss
val_loader = DataLoader(testset, batch_size=256, shuffle=True)
    
# List to store average loss per epoch
epoch_losses = []
epoch_precisions = []

run_clpso(model, val_loader, fine_tune_epochs=50, num_particles=100)

Process SpawnPoolWorker-106:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'parallel_evaluate' on <module '__main__' (built-in)>
Process SpawnPoolWorker-105:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/

  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'parallel_evaluate' on <module '__main__' (built-in)>
Process SpawnPoolWorker-117:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'parallel_evaluate' on <module '__main__' (built-in)>
Process SpawnPoolWorker-118

Process SpawnPoolWorker-128:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'parallel_evaluate' on <module '__main__' (built-in)>
Process SpawnPoolWorker-129:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/

Process SpawnPoolWorker-140:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'parallel_evaluate' on <module '__main__' (built-in)>
Process SpawnPoolWorker-141:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/

Process SpawnPoolWorker-152:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'parallel_evaluate' on <module '__main__' (built-in)>
Process SpawnPoolWorker-153:
Traceback (most recent call last):
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/school/GPTEngineer_conda/Miniconda3/envs/CNN/lib/python3.8/

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'particles': particles,
    'velocities': velocities,
    'personal_best_positions': personal_best_positions,
    'personal_best_scores': personal_best_scores,
    'global_best_position': global_best_position,
    'global_best_score': global_best_score,
    'epoch_losses': epoch_losses,
    'epoch_precisions': epoch_precisions
}, '/school/intelligence_coursework/Trained_networks/model_clpso_checkpoint_20_swarm.pth')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(epoch_losses, label='Training Loss')
plt.plot(epoch_precisions, label='Average Precision')
plt.xlabel('Epochs')

plt.ylabel('Metrics')
plt.title('Training Loss and Average Precision Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [28]:
correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[classes[label.item()]] += 1
            total_pred[classes[label.item()]] += 1
            
print('Accuracy of the network on the 10 000 test images: %f %%' % (100 * correct / total))

for classname, correct_count in correct_pred.items():
    accuracy = 100* float(correct_count)/ total_pred[classname]
    print(f'Accuracy for: {classname:5s} is {accuracy:0.1f} %')

Accuracy of the network on the 10 000 test images: 79.100000 %
Accuracy for: airplane is 80.1 %
Accuracy for: automobile is 88.2 %
Accuracy for: bird  is 68.9 %
Accuracy for: cat   is 62.6 %
Accuracy for: deer  is 83.0 %
Accuracy for: dog   is 70.4 %
Accuracy for: frog  is 85.0 %
Accuracy for: horse is 77.9 %
Accuracy for: ship  is 89.5 %
Accuracy for: truck is 85.4 %


In [ ]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        all_labels.append(labels.cpu().numpy())
        all_predictions.append(predicted.cpu().numpy())

# Concatenate all batches
all_labels = np.concatenate(all_labels)
all_predictions = np.concatenate(all_predictions)

#Assigning classes
def indices_to_class_names(indices, class_names):
    return [class_names[i] for i in indices]

# Apply the mapping to all labels and predictions
all_labels_names = indices_to_class_names(all_labels, classes)
all_predictions_names = indices_to_class_names(all_predictions, classes)

# Calculate metrics using the numerical indices
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')

# Print metrics
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix with Class Names
conf_matrix = confusion_matrix(all_labels_names, all_predictions_names)
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()